In [2]:
from scipy.optimize import minimize
from scipy.optimize import Bounds
import pandas as pd
import numpy as np
import os
import json

In [6]:
data_dir = '../data'

In [98]:
nutrition_profile_df = pd.DataFrame(index=['RDI', 'UL', 'target_fac', 'target_mask'], data={
    "Calories (kcal)": [2000, 2200, 1.0, True],
    "Carbohydrates (g)": [279, 300, 1.0, True],
    "Protein (g)": [104, 300, 1.0, True],
    "Total Fat (g)": [57, 70, 1.0, True],
    "Saturated Fat (g)": [0, 0, 1.0, False],
    "Monounsaturated Fat (g)": [0, 0, 1.0, False],
    "Polyunsaturated Fat (g)": [0, 0, 1.0, False],
    "Omega-3 Fatty Acids (g)": [8.5, np.NaN, 1.2, True],
    "Omega-6 Fatty Acids (g)": [12, np.NaN, 1.2, True],
    "Total Fiber (g)": [38, np.NaN, 1.2, True],
    "Soluble Fiber (g)": [3, np.NaN, 1.2, True],
    "Insoluble Fiber (g)": [30, np.NaN, 1.2, True],
    "Cholesterol (mg)": [0, np.NaN, 1.0, False],
    "Vitamin A (IU)": [3000, 10000, 1.2, True],
    "Vitamin B6 (mg)": [1.3, 100, 1.2, True],
    "Vitamin B12 (ug)": [2.4, np.NaN, 1.2, True],
    "Vitamin C (mg)": [90, 2000, 1.2, True],
    "Vitamin D (IU)": [600, 4000, 1.2, True],
    "Vitamin E (IU)": [20, 1000, 1.2, True],
    "Vitamin K (ug)": [120, np.NaN, 1.2, True],
    "Thiamin (mg)": [1.2, np.NaN, 1.2, True],
    "Riboflavin (mg)": [1.3, np.NaN, 1.2, True],
    "Niacin (mg)": [16, 41, 1.2, True],
    "Folate (ug)": [400, 1000, 1.2, True],
    "Pantothenic Acid (mg)": [5, np.NaN, 1.2, True],
    "Biotin (ug)": [30, np.NaN, 1.2, True],
    "Choline (mg)": [550, 3500, 1.2, True],
    "Calcium (g)": [1, 2.5, 1.2, True],
    "Chloride (g)": [2.3, np.NaN, 1.2, True],
    "Chromium (ug)": [35, np.NaN, 1.0, True],
    "Copper (mg)": [0.9, 10, 1.0, True],
    "Iodine (ug)": [150, 1100, 1.2, True],
    "Iron (mg)": [8, 45, 1.2, True],
    "Magnesium (mg)": [420, np.NaN, 1.2, True], 
    "Manganese (mg)": [2.3, 11, 1.0, True],
    "Molybdenum (ug)": [45, 2000, 1.2, True],
    "Phosphorus (g)": [0.7, 4, 1.2, True],
    "Potassium (g)": [4.7, np.NaN, 1.2, True],
    "Selenium (ug)": [55, 400, 1.2, True],
    "Sodium (g)": [1.5, 2.3, 1.2, True],
    "Sulfur (g)": [0.8, 2, 1.2, True],
    "Zinc (mg)": [11, 40, 1.2, True]
})
#nutrition_profile_df = nutrition_profile_df.loc[4] = np.zeros(nutrition_profile_df.shape[1]).reshape(1,42), index=['target']))
#np.zeros(nutrition_profile_df.shape[1]).reshape(1,42)
#nutrition_profile_df.shape
#pd.DataFrame(np.zeros(nutrition_profile_df.shape[1]), index=['target'])
#nutrition_profile_df.loc['target'] = nutrition_profile_df.loc['RDI'] * nutrition_profile_df.loc['target_fac']
#nutrition_profile_df.append(pd.Series(np.zeros(nutrition_profile_df.shape[1]).reshape(1,42)))
nutrition_profile_df.append(pd.DataFrame(np.zeros(nutrition_profile_df.shape[1]).reshape(1,42), columns=nutrition_profile_df.columns, index=['target']))
#nutrition_profile_df.head(5)

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),...,Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg)
RDI,2000,279,104,57,0,0,0,8.5,12,38,...,8,420,2.3,45,0.7,4.7,55,1.5,0.8,11
UL,2200,300,300,70,0,0,0,NaN,NaN,NaN,...,45,NaN,11,2000,4,NaN,400,2.3,2,40
target_fac,1,1,1,1,1,1,1,1.2,1.2,1.2,...,1.2,1.2,1,1.2,1.2,1.2,1.2,1.2,1.2,1.2
target_mask,True,True,True,True,False,False,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
target,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
nutrition_profile_df.loc['target'] = nutrition_profile_df.loc['RDI'] * nutrition_profile_df.loc['target_fac']
nutrition_profile_df.head(5)

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),...,Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg)
RDI,2000,279,104,57,0,0,0,8.5,12,38,...,8,420,2.3,45,0.7,4.7,55,1.5,0.8,11
UL,2200,300,300,70,0,0,0,NaN,NaN,NaN,...,45,NaN,11,2000,4,NaN,400,2.3,2,40
target_fac,1,1,1,1,1,1,1,1.2,1.2,1.2,...,1.2,1.2,1,1.2,1.2,1.2,1.2,1.2,1.2,1.2
target_mask,True,True,True,True,False,False,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
target,2000,279,104,57,0,0,0,10.2,14.4,45.6,...,9.6,504,2.3,54,0.84,5.64,66,1.8,0.96,13.2


In [103]:

field_map = {
    'Name': 8,
    'Container Size (g)': 12,
    'Cost ($)': 16,
    'Source': 18,
    'Link': 20,
    'Calories (kcal)': 32,
    'Carbohydrates (g)': 35,
    "Protein (g)": 38,
    "Total Fat (g)": 41,
    "Saturated Fat (g)": 44,
    "Monounsaturated Fat (g)": 47,
    "Polyunsaturated Fat (g)": 50,
    "Omega-3 Fatty Acids (g)": 53,
    "Omega-6 Fatty Acids (g)": 56,
    "Total Fiber (g)": 59,
    "Soluble Fiber (g)": 62,
    "Insoluble Fiber (g)": 65,
    "Cholesterol (mg)": 68,
    "Vitamin A (IU)": 116,
    "Vitamin B6 (mg)": 119,
    "Vitamin B12 (ug)": 122,
    "Vitamin C (mg)": 125,
    "Vitamin D (IU)": 128,
    "Vitamin E (IU)": 131,
    "Vitamin K (ug)": 134,
    "Thiamin (mg)": 137,
    "Riboflavin (mg)": 140,
    "Niacin (mg)": 143,
    "Folate (ug)": 146,
    "Pantothenic Acid (mg)": 149,
    "Biotin (ug)": 152,
    "Choline (mg)": 155,
    "Calcium (g)": 71,
    "Chloride (g)": 74,
    "Chromium (ug)": 77,
    "Copper (mg)": 80,
    "Iodine (ug)": 83,
    "Iron (mg)": 86,
    "Magnesium (mg)": 89, 
    "Manganese (mg)": 92,
    "Molybdenum (ug)": 95,
    "Phosphorus (g)": 98,
    "Potassium (g)": 101,
    "Selenium (ug)": 104,
    "Sodium (g)": 107,
    "Sulfur (g)": 110,
    "Zinc (mg)": 113
}

ingredients = {}
for file in os.listdir(f'{data_dir}/raw'):

    if not file.endswith('.txt'):
        continue
    #print(f"name of file: {file}")        
    lines = open(f'{data_dir}/raw/{file}', 'r').readlines()

    name = lines[8].strip()
    ingred = {}
    
    for k, v in field_map.items():
        if k.endswith(')'):
            ingred[k] = float(lines[v].strip())
        else:
            ingred[k] = lines[v].strip()
    ingredients[name] = ingred

open(f'{data_dir}/interim/ingredients.json', 'w').write(json.dumps(ingredients, indent=4))
#print(str(ingredients))


31865

In [104]:
ingredient_labels=list(ingredients[list(ingredients.keys())[0]].keys())[5:]
ingredient_names=list(ingredients.keys())
data = {}
for label in ingredient_labels:
    data[label]=[]
    
for name in ingredient_names:
    for label in ingredient_labels:
        data[label].append(ingredients[name][label])
        
ingredients_df = pd.DataFrame(index=ingredient_names, data=data)
ingredients_df.to_json(open(f'{data_dir}/interim/ingredients_df.json','w'))
ingredients_df.head(5)

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),...,Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg)
Whey protein,393.0,6.0,80.0,7.0,0.00,0.00,0.00,0.000,0.000,0.0,...,1.2,160.0,0.0,0.0,0.2,0.48,0.0,0.280,0.0,0.0
Ascorbic acid,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,0.0
Lecithin granules,700.0,10.0,0.0,50.0,15.00,0.00,35.00,0.000,0.000,0.0,...,3.6,0.0,0.0,0.0,3.0,1.60,0.0,0.000,0.0,0.0
Alpha-galactosidase enzyme (Beano),0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000,0.0,0.0
Guar gum,332.0,77.3,4.6,0.5,0.06,0.22,0.07,0.001,0.067,77.3,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.125,0.0,0.0


In [124]:
def objective(recipe):
    # recipe: contains the proportion of each ingredient measured in fractions of 100g
    
    # Find the total amount of all nutrients for the given recipe
    total = np.zeros(len(ingredients_df.values[0]))
    for i, amount in [ (i, np.transpose(recipe)[i]) for i in range(len(recipe))]:
        print(str(total))
        ingredient = ingredients_df.values[i]
        scaled = ingredient * amount
        total = total + scaled

    
    # Find the difference between the total and the target
    
    # Calculate the sum of the differences
        

In [125]:
#rec = np.array(np.ones(42))
objective(rec)
#np.transpose(rec)[1]

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[3.93e+02 6.00e+00 8.00e+01 7.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 5.00e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 1.20e+00 1.60e+02 0.00e+00 0.00e+00 2.00e-01 4.80e-01 0.00e+00 2.80e-01
 0.00e+00 0.00e+00]
[3.93e+02 6.00e+00 8.00e+01 7.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 1.00e+03 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 5.00e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 1.20e+00 1.60e+02 0.00e+00 0.00e+00 2.00e-01 4.80e-01 0.00e+00 2.80e-01
 0.00e+00 0.00e+00]
[1.093e+03 1.600e+01 8.000e+01 5.700e+01 1.500e+01 0.000e+00 3.500e+01
 0.000e+00 0.000e+00 0.000e+00

IndexError: index 20 is out of bounds for axis 0 with size 20

In [78]:
a = ingredients_df.values[0]

In [79]:
np.absolute(a * -1)

array([3.93e+02, 6.00e+00, 8.00e+01, 7.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 5.00e-01, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 1.20e+00, 1.60e+02, 0.00e+00, 0.00e+00,
       2.00e-01, 4.80e-01, 0.00e+00, 2.80e-01, 0.00e+00, 0.00e+00])

In [80]:
np.zeros(2)

array([0., 0.])